# Few-shot prompt templates

In [92]:
!pip install --upgrade --quiet langchain langchain-openai

In [28]:
import os
os.environ["OPENAI_API_KEY"]      = "sk-*****************************************************"

## ChatOpenAI

In [93]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
    temperature=0,
    max_tokens=2048,
    model_name="gpt-3.5-turbo",
)

question = "대한민국의 수도는 뭐야?"

print(f"[답변]: {llm.predict(question)}")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `predict` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


[답변]: 대한민국의 수도는 서울이야.


## PromptTemplate

In [95]:
from langchain.prompts import PromptTemplate

template = "{country}의 수도는 뭐야?"

prompt = PromptTemplate.from_template(template=template)

In [96]:
chain = prompt | llm

response = chain.invoke({"country": "korea"})

print(response.content)

대한민국의 수도는 서울이다.


## LLM Chain(Legacy)

In [97]:
from langchain.chains import LLMChain

llm_chain = LLMChain(prompt=prompt, llm=llm)

### run()

In [98]:
llm_chain.run(country="대만")

'대만의 수도는 타이베이(Taipei)입니다.'

In [99]:
llm_chain.predict(country="대한민국")

'대한민국의 수도는 서울이야.'

### apply()

In [101]:
input_list = [{"country": "대만"}, {"country": "중국"}, {"country": "몽골"}]
llm_chain.apply(input_list)

[{'text': '대만의 수도는 타이베이입니다.'},
 {'text': '중국의 수도는 베이징(北京)입니다.'},
 {'text': '몽골의 수도는 울란바토르입니다.'}]

In [102]:
result = llm_chain.apply(input_list)

for res in result:
    print(res["text"].strip())


대만의 수도는 타이베이(Taipei)입니다.
중국의 수도는 베이징(北京)입니다.
몽골의 수도는 울란바토르입니다.


### generate()

In [103]:
input_list = [{"country": "대만"}, {"country": "중국"}, {"country": "몽골"}]

generated_result = llm_chain.generate(input_list)
print(generated_result)

generations=[[ChatGeneration(text='대만의 수도는 타이베이(Taipei)입니다.', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='대만의 수도는 타이베이(Taipei)입니다.', response_metadata={'finish_reason': 'stop', 'logprobs': None}))], [ChatGeneration(text='중국의 수도는 베이징(北京)입니다.', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='중국의 수도는 베이징(北京)입니다.', response_metadata={'finish_reason': 'stop', 'logprobs': None}))], [ChatGeneration(text='몽골의 수도는 울란바토르입니다.', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='몽골의 수도는 울란바토르입니다.', response_metadata={'finish_reason': 'stop', 'logprobs': None}))]] llm_output={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 55, 'total_tokens': 112}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_4f2ebda25a'} run=[RunInfo(run_id=UUID('2fa0d357-8896-44a3-a515-7c78968a51f7')), RunInfo(run_id=UUID('21521b26-3dba-4511-b814-1a2b05ddf040')), RunInfo(run_id=UUID('597ab

In [104]:
generated_result.generations

[[ChatGeneration(text='대만의 수도는 타이베이(Taipei)입니다.', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='대만의 수도는 타이베이(Taipei)입니다.', response_metadata={'finish_reason': 'stop', 'logprobs': None}))],
 [ChatGeneration(text='중국의 수도는 베이징(北京)입니다.', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='중국의 수도는 베이징(北京)입니다.', response_metadata={'finish_reason': 'stop', 'logprobs': None}))],
 [ChatGeneration(text='몽골의 수도는 울란바토르입니다.', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='몽골의 수도는 울란바토르입니다.', response_metadata={'finish_reason': 'stop', 'logprobs': None}))]]

In [105]:
generated_result.llm_output

{'token_usage': {'completion_tokens': 57,
  'prompt_tokens': 55,
  'total_tokens': 112},
 'model_name': 'gpt-3.5-turbo',
 'system_fingerprint': 'fp_4f2ebda25a'}

In [106]:
generated_result.run

[RunInfo(run_id=UUID('2fa0d357-8896-44a3-a515-7c78968a51f7')),
 RunInfo(run_id=UUID('21521b26-3dba-4511-b814-1a2b05ddf040')),
 RunInfo(run_id=UUID('597abd39-25f2-4607-b923-2dbda3850db4'))]

In [107]:
for gen in generated_result.generations:
    print(gen[0].text.strip())

대만의 수도는 타이베이(Taipei)입니다.
중국의 수도는 베이징(北京)입니다.
몽골의 수도는 울란바토르입니다.


## Define two or more variables in a template

In [108]:
template = "What is the time difference between {area1} and {area2}?"

prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['area1', 'area2'], template='What is the time difference between {area1} and {area2}?')

In [109]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [110]:
print(llm_chain.run(area1="서울", area2="베이징"))

The time difference between Seoul, South Korea and Beijing, China is one hour. Seoul is one hour ahead of Beijing.


In [111]:
input_list = [
    {"area1": "서울", "area2": "뉴욕"},
    {"area1": "뉴욕", "area2": "울란바토르"},
    {"area1": "서울", "area2": "울란바토르"},
]

# 반복문으로 결과 출력
result = llm_chain.apply(input_list)
for res in result:
    print(res["text"].strip())

The time difference between 서울 (Seoul) and 뉴욕 (New York) is 14 hours. 서울 is 14 hours ahead of 뉴욕.
The time difference between New York and Ulaanbaatar is 12 hours. Ulaanbaatar is 12 hours ahead of New York.
The time difference between Seoul, South Korea and Ulaanbaatar, Mongolia is 1 hour. Seoul is 1 hour ahead of Ulaanbaatar.
